In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from NoiseKD import Teacher, ToyTransformer, slm_init_config, slm_model_config, count_parameters

In [2]:
#SLM = SimpleLanguageModel(**slm_init_config) #THIS GRADIENT ACCUMULATES!!!!BUGGGGGG 

In [3]:
embedding_dim = 16
num_heads = 8
hidden_dim  = 11
num_layers = 2
dropout = 0.1
vocab_size = 80
class_num = vocab_size
batch_size = 50
sequence_length = 160

In [4]:
TT = ToyTransformer(vocab_size, embedding_dim, num_heads, hidden_dim, num_layers, dropout,sequence_length)
teacher_toy = Teacher(TT,(sequence_length,))

In [5]:
teacher_toy.load_state_dict('good_toy_5.pth')
#teacher_toy.model

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


In [6]:
teacher_toy.linearize_embedding()
#teacher_toy.model

Teacher Embedding Linearized!


In [ ]:
#we want to have a configured teacher, then be able to duplicate it with the embedding layer converted to something
#that can take floats.
train_args = { 'val_train' : "train"
                      , 'n' : 1000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size'] ##this should be inferred in the method.
                      , 'std': 1.0
        }
teacher_toy.generate_data(**train_args)

val_args = { 'val_train' : "val"
                      , 'n' : 1000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_toy.generate_data(**val_args)

In [11]:
lin_val_args = { 'val_train' : "val"
                      , 'n' : 100
                      , 'dist_type' : 'normal_clipped_inverse'
        }
teacher_toy.generate_data(**lin_val_args)

In [12]:
teacher_toy.val_inputs

tensor([[[0.0000, 0.3923, 0.5651,  ..., 0.4751, 0.0420, 0.1020],
         [0.6553, 0.4580, 0.6098,  ..., 0.2148, 0.7105, 0.5373],
         [0.2807, 0.5236, 0.6551,  ..., 0.4721, 0.7736, 0.6352],
         ...,
         [0.4922, 0.4807, 0.5986,  ..., 0.8236, 0.4456, 0.2389],
         [0.4042, 0.4483, 0.6076,  ..., 0.2904, 0.5027, 0.2318],
         [0.4612, 0.4757, 0.1363,  ..., 0.2545, 0.5905, 0.1655]],

        [[0.6465, 0.5841, 0.6192,  ..., 0.5369, 0.5969, 0.8158],
         [0.5441, 0.6152, 0.4416,  ..., 0.9398, 0.6739, 0.4845],
         [0.3579, 1.0000, 0.4938,  ..., 0.5562, 0.4871, 0.0000],
         ...,
         [0.3428, 0.6276, 0.3583,  ..., 0.0000, 0.6080, 0.3671],
         [0.6180, 0.3862, 0.3719,  ..., 0.3041, 0.5790, 0.6470],
         [0.5926, 0.5293, 0.3705,  ..., 0.3968, 0.3862, 0.4139]],

        [[0.7075, 0.4595, 0.3828,  ..., 0.4934, 0.7570, 0.5298],
         [0.5745, 0.4999, 0.1969,  ..., 0.7156, 0.4552, 0.8030],
         [0.4113, 0.6515, 0.5642,  ..., 0.6872, 0.4792, 0.

In [ ]:
import numpy as np
gen_shape = (3, 4,7) 
samples = np.random.normal(0.5, 0.2, gen_shape)
samples = 1-np.clip(samples, 0, 1)
samples

In [ ]:
teacher_toy.val_inputs[0:10]

In [ ]:
def batch_one_hot(input_sequences, vocab_size):
    batch_size = input_sequences.size(0)
    max_seq_length = input_sequences.size(1)
    
    # Create a tensor to store the one-hot encodings
    one_hot_input = torch.zeros(batch_size, max_seq_length, vocab_size)
    
    # Use scatter_ to set the appropriate elements to 1 in each batch
    one_hot_input.scatter_(2, input_sequences.unsqueeze(2), 1)
    
    return one_hot_input


##test_data == torch.argmax(oh, dim=2)

In [ ]:
aa = (10,)
bb = (23,)
n = 100
gen_shape = (n,) + aa + bb
gen_shape

In [ ]:
oh = batch_one_hot(teacher_toy.val_inputs, vocab_size = 80)

In [ ]:
#oh[0:10].shape  torch.Size([10, 160, 80])
teacher_toy.model(oh[0:10]) #this works.  add vocab_size at the end.....i think i can get that from the embedding layer?


In [ ]:
def has_embedding_layer(model):
    for module in model.modules():
        if isinstance(module, nn.Embedding):
            return True
    return False

def Linearize_Embedding(embedding_layer):
    embedding_weight_tensor = embedding_layer.weight.detach() 
    shape = embedding_weight_tensor.shape
    vocab_size = shape[0]
    embedding_dim = shape[1]
    lin = nn.Linear(vocab_size,embedding_dim, bias = False)
    #print(lin.weight.shape)
    #print(embedding_weight_tensor.shape)
    lin.weight = nn.Parameter(embedding_weight_tensor.T) #not sure about this transpose
    return lin

def Linearize_Model_Embedding(model):
    if not has_embedding_layer(model):
        raise ValueError("Your model should have an Embedding layer, or maybe try an art degree.")
    embedding_layer = model.embedding
    linearized_embedding = Linearize_Embedding(embedding_layer)
    model_copy = copy.deepcopy(model)
    model_copy.embedding = linearized_embedding 
    return model_copy

L = Linearize_Embedding(embedding_layer)
# i want a function in KDNoise that takes a model with an embedding layer, and linearizes it. no, add it as an attribute maybe?
#so, the model should have an embedding layer, if it does you make a new instance of the model, and store it in like
#Teacher.model_linearized.  and, it needs to be able to be instantiated with the state_dict of the of teacher.  

In [ ]:
import copy
embedding_layer = teacher_toy.model.embedding
L = Linearize_Embedding(embedding_layer)
copy_TT = copy.deepcopy(TT)
copy_TT.embedding = L #swap out the embedding
#TT,copy_TT

In [ ]:
 #should just set self.model i think.   

In [ ]:
embedding_layer(test_data).shape,L(oh).shape
#normal takes the test data (ints), linearized takes onehotted data.

In [ ]:
#embedding_layer(test_data),L(oh)

In [ ]:
torch.sum((L(oh) - embedding_layer(test_data))**2) #ha.  it was the bias.  

In [ ]:
nn.Parameter(embedding_weight_tensor) == L.weight.T #init works. not sure about having a bias in linear layer tho...

In [ ]:
TT(test_data[0:10]) #,test_data[0:10] ##accumulating gradient!!!!!!

In [ ]:
oh = batch_one_hot(test_data, vocab_size = 80) #i do this above, this is just so i can remeber what it is doing
copy_TT(oh[0:10]) #now it takes one hot, not indexes.

In [ ]:
torch.sum((TT(test_data) - copy_TT(oh))**2) #asdflkajsdglfonsdg;aogh so the parameter layer is working, but the final bit isnt

In [ ]:
# final check, is the batch bug fixed"
TT(test_data[0:10])[3:4],TT(test_data[3:4]) ##just floating point!!!!

In [ ]:
copy_TT(oh[0:10])[3:4],copy_TT(oh[3:4])